# RPLib Problem 0001 - Baseline

Provides the baseline version to rankability problem 0001. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import copy
import os

import pandas as pd
import numpy as np

from scipy.stats import pearsonr

from tqdm import tqdm
from joblib import Parallel, delayed
import joblib
import itertools
from pathlib import Path
from sklearn.pipeline import Pipeline

from IPython.display import display, Markdown, Latex

**All packages are relative to the home directory of the user**

In [3]:
home = str(Path.home())

**Import the main rankability package**

In [4]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [5]:
sys.path.insert(0,"%s/RPLib"%home)
import pyrplib

**Load the problem information**

In [6]:
problem = joblib.load("generate.joblib.z")

## Explore and setup the problem

In [7]:
problem.keys()

dict_keys(['description', 'target', 'data', 'other'])

In [8]:
print(problem["description"])

First representative example for the rankability library. Built around the study of NCAA Men's Basketball league.


In [9]:
problem['target']

,days_to_subtract1,days_to_subtract2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,rankings1,rankings2,top10_jaccard,top10_tau,days_diff
0,28,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.388889,7
1,21,35,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.636364,-0.111111,14
2,14,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.055556,21
3,7,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.000000,28
4,21,28,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.722222,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,14,28,all,madness,0,0,0.1,Massey,2018,Cincinnati 8.0 Duke 2.0 Ka...,Cincinnati 6.0 Duke 4.0 Ka...,0.800000,0.222222,14
1696,7,28,all,madness,0,0,0.1,Massey,2018,Cincinnati 7.0 Duke 2.0 Ka...,Cincinnati 6.0 Duke 4.0 Ka...,0.800000,-0.111111,21
1697,14,21,all,madness,0,0,0.1,Massey,2018,Cincinnati 8.0 Duke 2.0 Ka...,Cincinnati 7.0 Duke 3.0 Ka...,0.800000,0.222222,7
1698,7,21,all,madness,0,0,0.1,Massey,2018,Cincinnati 7.0 Duke 2.0 Ka...,Cincinnati 7.0 Duke 3.0 Ka...,0.800000,0.222222,14


In [10]:
problem['data'].keys()

dict_keys(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [11]:
problem['data']['2002'].keys()

dict_keys(['days_to_subtract=35', 'days_to_subtract=28', 'days_to_subtract=21', 'days_to_subtract=14', 'days_to_subtract=7'])

**Create easier to reference variables**

In [12]:
years = list(problem['data'].keys())
days_to_subtract_keys = list(problem['data'][years[0]].keys())
remaining_games = problem['other']['remaining_games']
madness_teams = problem['other']['madness_teams']
best_df = problem['other']['best_df']
top_k = problem['other']['top_k']
#target_column = f"top{top_k}_intersection"
feature_names = problem['other']['feature_names']

In [13]:
days_to_subtract_keys

['days_to_subtract=35',
 'days_to_subtract=28',
 'days_to_subtract=21',
 'days_to_subtract=14',
 'days_to_subtract=7']

In [14]:
target = problem['target']

In [15]:
target

,days_to_subtract1,days_to_subtract2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,rankings1,rankings2,top10_jaccard,top10_tau,days_diff
0,28,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.388889,7
1,21,35,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.636364,-0.111111,14
2,14,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.055556,21
3,7,35,all,madness,0,0,0.1,Colley,2002,Alabama 6.0 Arizona 7.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,0.000000,28
4,21,28,all,madness,0,0,0.1,Colley,2002,Alabama 5.0 Arizona 9.0 Cincinnati...,Alabama 6.0 Arizona 7.0 Cincinnati...,0.800000,-0.722222,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,14,28,all,madness,0,0,0.1,Massey,2018,Cincinnati 8.0 Duke 2.0 Ka...,Cincinnati 6.0 Duke 4.0 Ka...,0.800000,0.222222,14
1696,7,28,all,madness,0,0,0.1,Massey,2018,Cincinnati 7.0 Duke 2.0 Ka...,Cincinnati 6.0 Duke 4.0 Ka...,0.800000,-0.111111,21
1697,14,21,all,madness,0,0,0.1,Massey,2018,Cincinnati 8.0 Duke 2.0 Ka...,Cincinnati 7.0 Duke 3.0 Ka...,0.800000,0.222222,7
1698,7,21,all,madness,0,0,0.1,Massey,2018,Cincinnati 7.0 Duke 2.0 Ka...,Cincinnati 7.0 Duke 3.0 Ka...,0.800000,0.222222,14


## Create D matrices

In [16]:
best_df

,days_to_subtract,Method,domain,range,direct_thres,spread_thres,weight_indirect
0,7,Colley,all,madness,0,0,0.1
1,7,Massey,all,madness,0,0,0.1
2,14,Colley,all,madness,0,0,0.1
3,14,Massey,all,madness,0,0,0.1
4,21,Colley,all,madness,0,0,0.1
5,21,Massey,all,madness,0,0,0.1
6,28,Colley,all,madness,0,0,0.1
7,28,Massey,all,madness,0,0,0.1
8,35,Colley,all,madness,0,0,0.1
9,35,Massey,all,madness,0,0,0.1


In [17]:
#Ds = process(problem['data'],problem['target'],best_df)

In [18]:
#Ds

In [19]:
#Ds.iloc[[0,-1]]

In [20]:
#Ds.loc['2002',"D"][0][0]

In [21]:
#Ds.loc['2002',"D"][0][1]

In [22]:
#Ds.index.names

In [23]:
rankings = {}
dt = target.set_index(['days_to_subtract2','Year','direct_thres','spread_thres','weight_indirect','range','Method'])
for index in dt.index:
    rankings[index] = dt.loc[index].iloc[0]['rankings2']
dt = target.set_index(['days_to_subtract1','Year','direct_thres','spread_thres','weight_indirect','range','Method'])
for index in dt.index:
    rankings[index] = dt.loc[index].iloc[0]['rankings1']

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys


In [24]:
rankings_df = pd.DataFrame(rankings).T

### Compute the features

In [26]:
index_cols = ["Year","days_to_subtract_key","direct_thres","spread_thres","weight_indirect","range","Method"]
#process_pipe = Pipeline([('compute_D', pyrplib.transformers.ProcessTransformer(index_cols, days_to_subtract_keys, years, 'madness', madness_teams, best_df))])
#process_pipe.fit(problem['data'])
#Ds = process_pipe.transform(problem['data'])
feature_columns = ["delta_lop","delta_hillside","nfrac_xstar_lop","nfrac_xstar_hillside","diameter_lop","diameter_hillside"]
#feature_pipe = Pipeline([('create_features', pyrplib.transformers.CreateFeaturesTransformer(feature_columns, rankings_df, top_k))])
#feature_pipe.fit(Ds)
#X = feature_pipe.transform(Ds)
process_feature_pipe = Pipeline([('compute_D', pyrplib.transformers.ProcessTransformer(index_cols, days_to_subtract_keys, years, 'madness', madness_teams, best_df)),
                                ('create_features', pyrplib.transformers.CreateFeaturesTransformer(feature_columns, rankings_df, top_k))])
process_feature_pipe.fit(problem['data'])
X = process_feature_pipe.transform(problem['data'])

85it [03:52,  3.85s/it]
85it [03:54,  3.87s/it]
0it [00:00, ?it/s]

35 2002 0 0 0.1 madness Colley
Restricted license - for non-production use only - expires 2022-01-13


1it [00:00,  3.40it/s]

35 2002 0 0 0.1 madness Massey


2it [00:00,  3.50it/s]

...

168it [00:44,  3.66it/s]

7 2018 0 0 0.1 madness Colley


169it [00:44,  3.75it/s]

7 2018 0 0 0.1 madness Massey


170it [00:44,  3.81it/s]


In [27]:
# X = create_features(Ds,rankings_df,top_k)

In [28]:
X

delta_lop  \
Year days_to_subtract_key direct_thres spread_thres weight_indirect range   Method Construction              
2002 days_to_subtract=35  0            0            0.1             madness Colley Direct              0.0   
                                                                                   Indirect            4.0   
                                                                                   Both                0.4   
                                                                            Massey Direct              0.0   
                                                                                   Indirect            7.0   
...                                                                                                    ...   
2018 days_to_subtract=7   0            0            0.1             madness Colley Indirect            8.0   
                                                                                   Both                1.5   
                                                                            Massey Direct              1.0   
                                                                                   Indirect           16.0   
                                                                                   Both                3.1   

                                                                                                 delta_hillside  \
Year days_to_subtract_key direct_thres spread_thres weight_indirect range   Method Construction                   
2002 days_to_subtract=35  0            0            0.1             madness Colley Direct                   0.0   
                                                                                   Indirect                48.0   
                                                                                   Both                    56.0   
                                                                            Massey Direct                   0.0   
                                                                                   Indirect                62.0   
...                                                                                                         ...   
2018 days_to_subtract=7   0            0            0.1             madness Colley Indirect                62.0   
                                                                                   Both                    78.0   
                                                                            Massey Direct                   1.0   
                                                                                   Indirect                80.0   
                                                                                   Both                    90.0   

                                                                                                 nfrac_xstar_lop  \
Year days_to_subtract_key direct_thres spread_thres weight_indirect range   Method Construction                    
2002 days_to_subtract=35  0            0            0.1             madness Colley Direct                   56.0   
                                                                                   Indirect                 38.0   
                                                                                   Both                     12.0   
                                                                            Massey Direct                   56.0   
                                                                                   Indirect                 30.0   
...                                                                                                          ...   
2018 days_to_subtract=7   0            0            0.1             madness Colley Indirect                 14.0   
                                                                                   Both                      4.0   
          

## Refine the target dataset

In [29]:
target = problem['target'].groupby(['days_to_subtract1','days_to_subtract2','Method','Year','direct_thres','spread_thres','weight_indirect'])[feature_names].mean()
target

top10_jaccard  \
days_to_subtract1 days_to_subtract2 Method Year direct_thres spread_thres weight_indirect                  
7                 14                Colley 2002 0            0            0.1                   0.800000   
                                           2003 0            0            0.1                   0.800000   
                                           2004 0            0            0.1                   0.636364   
                                           2005 0            0            0.1                   1.000000   
                                           2006 0            0            0.1                   0.636364   
...                                                                                                  ...   
28                35                Massey 2014 0            0            0.1                   0.636364   
                                           2015 0            0            0.1                   0.800000   
                                           2016 0            0            0.1                   0.800000   
                                           2017 0            0            0.1                   1.000000   
                                           2018 0            0            0.1                   0.636364   

                                                                                           top10_tau  
days_to_subtract1 days_to_subtract2 Method Year direct_thres spread_thres weight_indirect             
7                 14                Colley 2002 0            0            0.1               0.277778  
                                           2003 0            0            0.1              -0.111111  
                                           2004 0            0            0.1               0.055556  
                                           2005 0            0            0.1              -0.555556  
                                           2006 0            0            0.1              -0.444444  
...                                                                                              ...  
28                35                Massey 2014 0            0            0.1               0.111111  
                                           2015 0            0            0.1              -0.222222  
                                           2016 0            0            0.1               0.166667  
                                           2017 0            0            0.1               0.111111  
                                           2018 0            0            0.1               0.055556  

[340 rows x 2 columns]

In [30]:
target.corr()

,top10_jaccard,top10_tau
top10_jaccard,1.000000,0.025604
top10_tau,0.025604,1.000000


In [31]:
X_for_join = X.copy().reset_index()
X_for_join['days_to_subtract1']= X_for_join['days_to_subtract_key'].str.replace("days_to_subtract=","").astype(float)
X_for_join.weight_indirect=0.1
X_for_join

,Year,days_to_subtract_key,direct_thres,spread_thres,weight_indirect,range,Method,Construction,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside,days_to_subtract1
0,2002,days_to_subtract=35,0,0,0.1,madness,Colley,Direct,0.0,0.0,56.0,70.0,28.0,35.0,35.0
1,2002,days_to_subtract=35,0,0,0.1,madness,Colley,Indirect,4.0,48.0,38.0,6.0,18.0,3.0,35.0
2,2002,days_to_subtract=35,0,0,0.1,madness,Colley,Both,0.4,56.0,12.0,8.0,6.0,4.0,35.0
3,2002,days_to_subtract=35,0,0,0.1,madness,Massey,Direct,0.0,0.0,56.0,68.0,28.0,34.0,35.0
4,2002,days_to_subtract=35,0,0,0.1,madness,Massey,Indirect,7.0,62.0,30.0,8.0,15.0,4.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2018,days_to_subtract=7,0,0,0.1,madness,Colley,Indirect,8.0,62.0,14.0,6.0,7.0,3.0,7.0
506,2018,days_to_subtract=7,0,0,0.1,madness,Colley,Both,1.5,78.0,4.0,12.0,2.0,4.0,7.0
507,2018,days_to_subtract=7,0,0,0.1,madness,Massey,Direct,1.0,1.0,32.0,44.0,16.0,22.0,7.0
508,2018,days_to_subtract=7,0,0,0.1,madness,Massey,Indirect,16.0,80.0,18.0,6.0,9.0,3.0,7.0


In [32]:
target

top10_jaccard  \
days_to_subtract1 days_to_subtract2 Method Year direct_thres spread_thres weight_indirect                  
7                 14                Colley 2002 0            0            0.1                   0.800000   
                                           2003 0            0            0.1                   0.800000   
                                           2004 0            0            0.1                   0.636364   
                                           2005 0            0            0.1                   1.000000   
                                           2006 0            0            0.1                   0.636364   
...                                                                                                  ...   
28                35                Massey 2014 0            0            0.1                   0.636364   
                                           2015 0            0            0.1                   0.800000   
                                           2016 0            0            0.1                   0.800000   
                                           2017 0            0            0.1                   1.000000   
                                           2018 0            0            0.1                   0.636364   

                                                                                           top10_tau  
days_to_subtract1 days_to_subtract2 Method Year direct_thres spread_thres weight_indirect             
7                 14                Colley 2002 0            0            0.1               0.277778  
                                           2003 0            0            0.1              -0.111111  
                                           2004 0            0            0.1               0.055556  
                                           2005 0            0            0.1              -0.555556  
                                           2006 0            0            0.1              -0.444444  
...                                                                                              ...  
28                35                Massey 2014 0            0            0.1               0.111111  
                                           2015 0            0            0.1              -0.222222  
                                           2016 0            0            0.1               0.166667  
                                           2017 0            0            0.1               0.111111  
                                           2018 0            0            0.1               0.055556  

[340 rows x 2 columns]

In [33]:
Xy = target.reset_index().set_index(['Method','days_to_subtract1','Year','direct_thres','spread_thres','weight_indirect']).join(X_for_join.set_index(['Method','days_to_subtract1','Year','direct_thres','spread_thres','weight_indirect'])).dropna()
Xy = Xy.reset_index()
Xy

,Method,days_to_subtract1,Year,direct_thres,spread_thres,weight_indirect,days_to_subtract2,top10_jaccard,top10_tau,days_to_subtract_key,range,Construction,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
0,Colley,7,2002,0,0,0.1,14,0.800000,0.277778,days_to_subtract=7,madness,Direct,1.0,0.0,50.0,54.0,25.0,27.0
1,Colley,7,2002,0,0,0.1,14,0.800000,0.277778,days_to_subtract=7,madness,Indirect,9.0,87.0,8.0,10.0,4.0,5.0
2,Colley,7,2002,0,0,0.1,14,0.800000,0.277778,days_to_subtract=7,madness,Both,2.0,104.0,6.0,10.0,3.0,5.0
3,Colley,7,2002,0,0,0.1,21,0.636364,-0.555556,days_to_subtract=7,madness,Direct,1.0,0.0,50.0,54.0,25.0,27.0
4,Colley,7,2002,0,0,0.1,21,0.636364,-0.555556,days_to_subtract=7,madness,Indirect,9.0,87.0,8.0,10.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,Massey,28,2017,0,0,0.1,35,1.000000,0.111111,days_to_subtract=28,madness,Indirect,11.0,80.0,16.0,10.0,8.0,5.0
1016,Massey,28,2017,0,0,0.1,35,1.000000,0.111111,days_to_subtract=28,madness,Both,2.1,92.0,12.0,0.0,6.0,-0.0
1017,Massey,28,2018,0,0,0.1,35,0.636364,0.055556,days_to_subtract=28,madness,Direct,2.0,2.0,64.0,64.0,31.0,31.0
1018,Massey,28,2018,0,0,0.1,35,0.636364,0.055556,days_to_subtract=28,madness,Indirect,7.0,84.0,16.0,2.0,8.0,1.0


## Process results

In [34]:
pairs_by_width = {}
for f1,f2 in itertools.combinations(X_for_join['days_to_subtract1'].unique().astype(int),2):
    if f2 < f1:
        f1,f2 = f2,f1
    width = f2-f1#round(100*(f2-f1))
    if width not in pairs_by_width:
        pairs_by_width[width] = []
    pairs_by_width[width].append((f1,f2))

In [35]:
pairs_by_width

{7: [(28, 35), (21, 28), (14, 21), (7, 14)],
 14: [(21, 35), (14, 28), (7, 21)],
 21: [(14, 35), (7, 28)],
 28: [(7, 35)]}

In [36]:
["days_to_subtract1","days_to_subtract2","Method","Construction"]+feature_columns

['days_to_subtract1',
 'days_to_subtract2',
 'Method',
 'Construction',
 'delta_lop',
 'delta_hillside',
 'nfrac_xstar_lop',
 'nfrac_xstar_hillside',
 'diameter_lop',
 'diameter_hillside']

In [37]:
#!sudo pip install pandas --upgrade

In [38]:
import altair as alt

index_cols = ['Method', 'Construction', 'days_to_subtract1', 'days_to_subtract2','width']
graph_dfs = {}
for target_column in feature_names:
    graph_df = pd.DataFrame(columns=index_cols+feature_columns).set_index(index_cols)

    for width in pairs_by_width.keys():
        summary = None
        for pair in pairs_by_width[width]:
            data = Xy.set_index(['days_to_subtract1','days_to_subtract2']).loc[pair].reset_index()
            for_corr = data.set_index(['Method','Construction',"days_to_subtract1","days_to_subtract2"])
            if summary is None:
                summary = pd.DataFrame(columns=["days_to_subtract1","days_to_subtract2","Method","Construction"]+feature_columns).set_index(list(for_corr.index.names))
            for ix in for_corr.index.unique():
                corr_results = for_corr.loc[ix][[target_column]+feature_columns].corr()
                target_corr_results = corr_results.loc[target_column].drop(target_column)
                target_corr_results.name = ix
                summary = summary.append(target_corr_results)

        graph_df1 = summary.reset_index()
        graph_df1['width'] = width
        graph_df1 = graph_df1.set_index(index_cols)
        graph_df = graph_df.append(graph_df1)
    graph_dfs[target_column]=graph_df

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  # This is added back by InteractiveShellApp.init_path()
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:16: PerformanceWarning: indexing past lexsort depth may impact performance.
  app.launch_new_instance()


In [39]:
for key in graph_dfs.keys():
    graph_dfs[key] = graph_dfs[key].reset_index()

In [ ]:
graph_dfs[key].head()

In [ ]:
for key in graph_dfs.keys():
    display(Markdown(f'## {key}'))
    graph_df = graph_dfs[key].melt(value_vars=feature_columns,id_vars=index_cols,value_name='Value',var_name='Feature')

    display(Markdown('### Colley'))
    g = alt.Chart(graph_df.set_index('Method').loc['Colley']).mark_bar().encode(
        x='width:N',
        y=alt.Y('average(Value)',scale=alt.Scale(domain=[-.6, .6])),
        row='Feature:N',
        color='Construction:N',
        column='Construction:N'
    )
    display(g)
    
    display(Markdown('### Massey'))
    g = alt.Chart(graph_df.set_index('Method').loc['Massey']).mark_bar().encode(
        x='width:N',
        y=alt.Y('average(Value)',scale=alt.Scale(domain=[-.6, .6])),
        row='Feature:N',
        color='Construction:N',
        column='Construction:N'
    )
    display(g)

### Old below this line

### Colley

In [ ]:
g = alt.Chart(graph_df.set_index('Method').loc['Massey']).mark_bar().encode(
    x='width:N',
    y=alt.Y('average(Value)',scale=alt.Scale(domain=[-.6, .6])),
    row='Feature:N',
    color='Construction:N',
    column='Construction:N'
)

### Massey

In [ ]:
g

## old below

## 0.6 to 0.7

(this was already broken)

In [ ]:
data = Xy.set_index(['frac1','frac2']).loc[(0.6,0.7)].reset_index()
for_corr = data.set_index(['Method','Construction'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

### 0.7 to 0.8

In [ ]:
data = Xy.set_index(['frac1','frac2']).loc[(0.7,0.8)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

### 0.8 to 0.9

In [ ]:
data = Xy.set_index(['frac1','frac2']).loc[(0.8,0.9)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

### 0.9 to 1.

In [ ]:
data = Xy.set_index(['frac1','frac2']).loc[(0.9,1.)].reset_index()
for_corr = data.set_index(['Method'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

In [ ]:
for_corr = data.set_index(['Method','direct_thres','spread_thres','weight_indirect'])
for_display = pd.DataFrame(columns=feature_columns+list(for_corr.index.names))
for_display.set_index(list(for_corr.index.names),inplace=True)
for ix in for_corr.index.unique():
    dt = for_corr.loc[ix][[target_column]+feature_columns].corr().loc[target_column,feature_columns]
    dt.name = ix
    for_display = for_display.append(dt)

In [ ]:
for_display.T

In [ ]:
print(for_display.T.to_latex())